# Real-world network

In [6]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [7]:
g = nx.read_gml("./RealNetworks/facebook_politician_pages.gml")

In [8]:
print(nx.info(g))

Graph with 5908 nodes and 41706 edges


<ipython-input-8-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [9]:
nx.is_connected(g)

True

In [10]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [11]:
from myTruss import mappingAndRelabeling

In [12]:
g_relabled = mappingAndRelabeling(g)

In [13]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 5908 nodes and 41706 edges
Graph with 5908 nodes and 41706 edges


<ipython-input-13-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-13-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [14]:
g = g_relabled

# Infer community structure with Louvain

In [15]:
nx.__version__

'2.8.4'

In [16]:
import pandas as pd
from os.path  import exists
import infomap as im

In [17]:
im.__version__

'2.3.0'

In [18]:
G = g

In [19]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [20]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [21]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [22]:
results_file = "./results/facebook_politician_pages-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/facebook_politician_pages-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/facebook_politician_pages-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/facebook_politician_pages-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [23]:
len(mec_ranks)

5908

In [24]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[177,
 308,
 746,
 703,
 749,
 1392,
 157,
 209,
 855,
 1395,
 853,
 752,
 140,
 80,
 70,
 754,
 74,
 41,
 34,
 13,
 1387,
 255,
 1229,
 747,
 1386,
 857,
 862,
 67,
 233,
 73,
 1265,
 50,
 941,
 1879,
 138,
 1231,
 1524,
 1369,
 147,
 2308,
 68,
 859,
 868,
 2318,
 1837,
 1777,
 945,
 1720,
 1527,
 508,
 40,
 1760,
 1774,
 743,
 257,
 1027,
 1435,
 1786,
 1773,
 1778,
 654,
 1780,
 1781,
 1782,
 1379,
 2123,
 732,
 554,
 1763,
 57,
 1436,
 772,
 932,
 2053,
 870,
 951,
 502,
 1783,
 1383,
 29,
 648,
 1876,
 1367,
 2052,
 1744,
 629,
 1776,
 1385,
 2317,
 1785,
 574,
 627,
 2313,
 618,
 1829,
 785,
 1841,
 777,
 561,
 36,
 83,
 72,
 2212,
 540,
 555,
 259,
 552,
 30,
 876,
 557,
 714,
 981,
 568,
 1830,
 1787,
 1373,
 948,
 721,
 28,
 21,
 577,
 669,
 256,
 810,
 1874,
 1534,
 1533,
 1979,
 108,
 1905,
 1432,
 64,
 1349,
 184,
 2315,
 625,
 723,
 634,
 228,
 2033,
 12,
 1134,
 505,
 164,
 202,
 516,
 551,
 713,
 560,
 1880,
 623,
 1820,
 1378,
 1434,
 2208,
 550,
 196,
 246,
 739,
 301

In [25]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [26]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [27]:
dict_map_eq_centrality

{177: 5908,
 308: 5907,
 746: 5906,
 703: 5905,
 749: 5904,
 1392: 5903,
 157: 5902,
 209: 5901,
 855: 5900,
 1395: 5899,
 853: 5898,
 752: 5897,
 140: 5896,
 80: 5895,
 70: 5894,
 754: 5893,
 74: 5892,
 41: 5891,
 34: 5890,
 13: 5889,
 1387: 5888,
 255: 5887,
 1229: 5886,
 747: 5885,
 1386: 5884,
 857: 5883,
 862: 5882,
 67: 5881,
 233: 5880,
 73: 5879,
 1265: 5878,
 50: 5877,
 941: 5876,
 1879: 5875,
 138: 5874,
 1231: 5873,
 1524: 5872,
 1369: 5871,
 147: 5870,
 2308: 5869,
 68: 5868,
 859: 5867,
 868: 5866,
 2318: 5865,
 1837: 5864,
 1777: 5863,
 945: 5862,
 1720: 5861,
 1527: 5860,
 508: 5859,
 40: 5858,
 1760: 5857,
 1774: 5856,
 743: 5855,
 257: 5854,
 1027: 5853,
 1435: 5852,
 1786: 5851,
 1773: 5850,
 1778: 5849,
 654: 5848,
 1780: 5847,
 1781: 5846,
 1782: 5845,
 1379: 5844,
 2123: 5843,
 732: 5842,
 554: 5841,
 1763: 5840,
 57: 5839,
 1436: 5838,
 772: 5837,
 932: 5836,
 2053: 5835,
 870: 5834,
 951: 5833,
 502: 5832,
 1783: 5831,
 1383: 5830,
 29: 5829,
 648: 5828,
 1876: 5

In [28]:
# Write Centrality
with open('./MapEquationCentralityResults/OSN-FacebookPolPages/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [29]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/OSN-FacebookPolPages/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [32]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True